In [7]:
from pathlib import Path
import re
from itertools import chain
from nltk.tokenize import wordpunct_tokenize, sent_tokenize
import random
import csv
from functools import reduce

DATA_FOLDER = Path('/data/NER')

zonazakona_sentences = DATA_FOLDER / 'zonazakona.txt'
legalforum_sentences = DATA_FOLDER / 'legalforum.txt'
yuristforum_sentences = DATA_FOLDER / 'yuristforum.txt'
find_people_sentences = DATA_FOLDER / 'find_people.txt'
vkdata_ads_sentences = DATA_FOLDER / 'vkdata_ads.txt'
vkdata_flat_sentences = DATA_FOLDER / 'vkdata_flat.txt'


regex_dataset = DATA_FOLDER / 'regex_dataset.tsv'

In [11]:
%%time

sentences = []
sources = []
for src, sent_file in [('zonazakona', zonazakona_sentences), 
                       ('legalforum', legalforum_sentences), 
                       ('yuristforum', yuristforum_sentences), 
                       ('find_people', find_people_sentences), 
                       ('vkdata_ads', vkdata_ads_sentences), 
                       ('vkdata_flat', vkdata_flat_sentences)]:
    with sent_file.open() as f:
        sents = list(f)
        sentences.extend(sents)
        sources.extend([src] * len(sents))

CPU times: user 2.95 s, sys: 638 ms, total: 3.59 s
Wall time: 3.59 s


In [12]:
len(sentences)

8241309

In [17]:
%%time

STREET_WORDS = re.compile(r'[^\w](ул|улиц|проезд|проспект|шоссе|пр\.|пр\-т)')
ADDRESS_WORDS = re.compile(r'[^\w](ул|д[^w]?[\d]+)[\.\ $]')

NUMBER_RE = re.compile(r'[\d]+([\/к]\d+)?')
INDEX_RE = re.compile(r'[^\d\w]\d{6}[^\d\w]')
ADDRESS_SHORT_RE = re.compile(r'[^\d\.](ул|кв|г|пр|ш|стр|лит|д|м|корп|пл)[\.\ ]')
ADDRESS_LONG_RE = re.compile(r'улиц[?\w<Б]')
MANY_SPACES_RE = re.compile(r'[\ \-\,]{2,}')
INLINE_DOTS_RE = re.compile(r'\ [\ \.]+\ ')

with regex_dataset.open('w') as f:
    c = csv.writer(f, delimiter='\t')    
    c.writerow(['addr', 'text', 'orig', 'source'])
    i = 0
    for src, s in zip(sources, sentences):
        k = ADDRESS_WORDS.search(s.replace('т.д.', ' ').replace('т.е.', ' ')) is not None \
            and NUMBER_RE.search(s) is not None
        line = reduce(lambda res, regex: regex.sub(' ', res), 
                      [INDEX_RE, NUMBER_RE, ADDRESS_LONG_RE, ADDRESS_SHORT_RE, MANY_SPACES_RE, INLINE_DOTS_RE], 
                      s.strip().replace('№', '')).strip()
        if len(line) < 2:
            continue
        c.writerow([int(k), line, s.strip(), src])
        i += int(k)
i
        

CPU times: user 2min 10s, sys: 1.57 s, total: 2min 11s
Wall time: 2min 14s


In [33]:
ex = ''' общем, за этот срок, посчитав размер неустойки по формуле страховая сумма (120000р.)
- направить либо представить лично письменное обращение по адресу: 153000, г. Иваново, улица Багаева, дом 27;
- записаться на прием по адресу: 153000, г. Иваново, улица Багаева, дом 27 каб.
находящийся по адресу: 103160, г. Москва, ул.Знаменка, 19, к-160
ГСП-3 125993 г. Москва, улица Большая Дмитровка, 15а
В результате неправомерных действий УУУУУУ мне причинен значительный материальный ущерб в сумме 930099 (Девятисот тридцати тысяч девяноста девяти) рублей 98 копеек.
Адрес: 142203, г. Серпухов, ул.Советская, д.80/8
Есть специализированные психологические центры, которые занимаются экспертизой по направлению суда в области сексульного насилия над детьми, например  Центр психолого-медико-социального сопровождения "ОЗОН" 107066, г. Москва, улица Нижняя Красносельская, дом 45/17 (м.
Управление по вопросам гражданства: 109240,г.Москва, Боярский переулок, д.4
959-89-43, 959-88-36, сайт labor.ru, адрес: 107078 москва, докучаев переулок, 12
'''.splitlines()

In [4]:
4800 / 60

80.0